In [ ]:
'''
Initial code was copied from:
https://github.com/jason71995/Keras-GAN-Library
'''

In [1]:
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D,GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose,Activation,BatchNormalization
from keras.optimizers import Adam

from keras.layers import Dense
from keras import initializers
from keras.layers.core import Dropout



noise_dim = 5

def build_generator(input_shape):

    generator = Sequential()
    
    generator.add(Dense(512,
                        input_dim = noise_dim))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(512))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(512))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))
    
    
    generator.add(Dense(512))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU(0.2))

    
    generator.add(Dense(105, activation='tanh'))
    return generator


def build_discriminator(input_shape):

    discriminator = Sequential()
    discriminator.add(Dense(512,
                            input_dim=105))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
        
    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    
    discriminator.add(Dense(1, activation='sigmoid'))

    return discriminator

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)

    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    pred_real = K.clip(pred_real,K.epsilon(),1-K.epsilon())
    pred_fake = K.clip(pred_fake,K.epsilon(),1-K.epsilon())

    d_loss = -(K.mean(K.log(pred_real)) + K.mean(K.log(1-pred_fake)))
    g_loss = -K.mean(K.log(pred_fake))

    # get updates of mean and variance in batch normalization layers
    d_updates = discriminator.get_updates_for([d_input])
    g_updates = generator.get_updates_for([noise])

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss],d_updates + d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_updates + g_training_updates)

    return d_train,g_train

Using TensorFlow backend.


In [3]:
# Loading MHAD data for action1, all persons and all repeatations of each person
from utils.data_loader import data_loader
data_object= data_loader(matlab_action_path='../gan/')
myData, mymin, mymax = data_object.actions_normalised([1], twoD_true_or_threeD_false=False)
myData.shape

(117562, 105)

In [5]:
import keras

#from gan_libs.DCGAN import build_generator, build_discriminator, build_functions
# from gan_libs.LSGAN import build_generator, build_discriminator, build_functions
# from gan_libs.SNGAN import build_generator, build_discriminator, build_functions
# from gan_libs.WGAN_GP import build_generator, build_discriminator, build_functions

from utils.common import set_gpu_config, predict_images
from utils.draw_pose import draw_pose
import numpy as np

set_gpu_config("0",0.5)

epoch = 500 + 1
steps = 1000
image_size = (1,1,105)
noise_size = (1,1,5)
batch_size = 16

x_train = myData

generator = build_generator(noise_size)
print(generator.summary())
discriminator = build_discriminator(image_size)
print(discriminator.summary())
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

for e in range(epoch):
    for s in range(steps):
        real_images = x_train[np.random.permutation(x_train.shape[0])[:batch_size]]
        real_images.shape = (batch_size,1,1,105)
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
     
    
    print ("[{0}/{1}] d_loss: {2:.4}, g_loss: {3:.4}".format(e, epoch, d_loss, g_loss))
    #generating a sample
    image = generator.predict(np.zeros(shape=(1,5)))
    image = np.array(image)
    draw_pose(image.reshape(105),'output',"e{0}".format(e))
        
    if e % 100 == 0:
        generator.save_weights("e{0}_generator.h5".format(e))
        discriminator.save_weights("e{0}_discriminator.h5".format(e))
        



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 512)               3072      
_________________________________________________________________
batch_normalization_5 (Batch (None, 512)               2048      
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               262656    
_________________________________________________________________
batch_normalization_6 (Batch (None, 512)               2048      
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               262656    
__________

[115/501] d_loss: 1.204, g_loss: 3.817
[116/501] d_loss: 1.176, g_loss: 2.621
[117/501] d_loss: 1.163, g_loss: 2.751
[118/501] d_loss: 1.133, g_loss: 2.701
[119/501] d_loss: 1.422, g_loss: 2.564
[120/501] d_loss: 1.197, g_loss: 2.739
[121/501] d_loss: 1.124, g_loss: 2.663
[122/501] d_loss: 1.07, g_loss: 2.742
[123/501] d_loss: 1.147, g_loss: 2.893
[124/501] d_loss: 1.248, g_loss: 2.727
[125/501] d_loss: 1.109, g_loss: 2.853
[126/501] d_loss: 1.325, g_loss: 3.346
[127/501] d_loss: 1.238, g_loss: 2.816
[128/501] d_loss: 1.433, g_loss: 3.774
[129/501] d_loss: 0.9829, g_loss: 2.682
[130/501] d_loss: 1.183, g_loss: 4.19
[131/501] d_loss: 1.041, g_loss: 3.264
[132/501] d_loss: 1.171, g_loss: 2.715
[133/501] d_loss: 1.402, g_loss: 3.584
[134/501] d_loss: 1.198, g_loss: 3.847
[135/501] d_loss: 1.081, g_loss: 2.665
[136/501] d_loss: 1.22, g_loss: 2.557
[137/501] d_loss: 1.095, g_loss: 3.851
[138/501] d_loss: 1.013, g_loss: 2.693
[139/501] d_loss: 1.056, g_loss: 3.469
[140/501] d_loss: 1.012, g_

[326/501] d_loss: 1.216, g_loss: 1.631
[327/501] d_loss: 1.136, g_loss: 1.968
[328/501] d_loss: 1.041, g_loss: 1.691
[329/501] d_loss: 1.188, g_loss: 1.745
[330/501] d_loss: 1.179, g_loss: 1.732
[331/501] d_loss: 1.256, g_loss: 1.857
[332/501] d_loss: 1.193, g_loss: 1.773
[333/501] d_loss: 1.436, g_loss: 2.098
[334/501] d_loss: 1.246, g_loss: 1.885
[335/501] d_loss: 1.135, g_loss: 2.438
[336/501] d_loss: 1.26, g_loss: 2.012
[337/501] d_loss: 1.071, g_loss: 1.684
[338/501] d_loss: 1.13, g_loss: 1.624
[339/501] d_loss: 0.9783, g_loss: 1.653
[340/501] d_loss: 1.239, g_loss: 1.914
[341/501] d_loss: 1.124, g_loss: 1.835
[342/501] d_loss: 1.245, g_loss: 1.692
[343/501] d_loss: 1.128, g_loss: 1.795
[344/501] d_loss: 0.9347, g_loss: 3.419
[345/501] d_loss: 1.221, g_loss: 2.13
[346/501] d_loss: 1.208, g_loss: 1.715
[347/501] d_loss: 1.317, g_loss: 1.854
[348/501] d_loss: 1.25, g_loss: 1.917
[349/501] d_loss: 1.318, g_loss: 1.765
[350/501] d_loss: 1.169, g_loss: 3.545
[351/501] d_loss: 1.369, g_

In [ ]:
generator.load_weights("e400_generator.h5".format(e))
discriminator.load_weights("e400_discriminator.h5".format(e))

In [ ]:
for i in range(100):
    x= np.array((i/100,0,0,0,0)).reshape(1,5)
    image = generator.predict(x)
    image = np.array(image)
    draw_pose(image.reshape(105),'output',"e{0}".format(i))